In [11]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [12]:
from google.colab import files
uploaded = files.upload()

Saving Scrap.csv to Scrap (1).csv


In [13]:
from google.colab import files
uploaded = files.upload()

Saving df_api (7).csv to df_api (7) (1).csv


In [14]:
df_api = pd.read_csv('df_api (7).csv')
df_scrap = pd.read_csv('Scrap.csv')

def norm(s):
    return (s.fillna('').str.strip().str.replace(r'\s+', ' ', regex=True).str.casefold())

df_scrap['key'] = norm(df_scrap['Artist']) + ' — ' + norm(df_scrap['Song'])
df_api['key']   = norm(df_api['Artist'])   + ' — ' + norm(df_api['Song'])
merged = (df_scrap.merge(df_api, on='key', how='inner').drop(columns=['key', 'Artist_y', 'Song_y']).rename(columns={'Artist_x': 'Artist', 'Song_x': 'Song'}))

def clean_days_in_top10(x):
    if pd.isna(x):
        return 0
    else:
        return x

def clean_number(x):
    if type(x) == str:
        return int(x.replace(',', ''))
    return int(x)

merged['Days in top10'] = merged['Days in top10'].apply(clean_days_in_top10)
merged['PkStreams'] = merged['PkStreams'].apply(clean_number)
merged['Total'] = merged['Total'].apply(clean_number)
merged

,Artist,Song,Days in top10,Peak,Days on peak,Days,PkStreams,Total,listeners,playcount,rank,track_url,artist_url,duration_min,duration,genre,genre_description,artist_listeners,artist_playcount
0,The Weeknd,Blinding Lights,446.0,1,82,2169,8453567,5037607641,2221161,36470257,52,https://www.last.fm/music/The+Weeknd/_/Blindin...,https://www.last.fm/music/The+Weeknd,3.333333,3 мин 20 сек,r&b,R&B is a musical genre that began to develop a...,5050479,1031011466
1,Harry Styles,As It Was,426.0,1,81,1272,16103849,3999274385,1867858,39479081,39,https://www.last.fm/music/Harry+Styles/_/As+It...,https://www.last.fm/music/Harry+Styles,2.750000,2 мин 45 сек,pop,Pop music is a broad-term for many different t...,3037938,381185399
2,Post Malone,Sunflower - Spider-Man: Into the Spider-Verse,213.0,1,16,2263,5033261,3554729660,1659244,24440448,214,https://www.last.fm/music/Post+Malone/_/Sunflo...,https://www.last.fm/music/Post+Malone,2.616667,2 мин 37 сек,rap,"Rap is a vocal style, usually coming together ...",3354307,312638678
3,The Neighbourhood,Sweater Weather,0.0,12,1,1988,3342205,3532412627,2750909,45328300,29,https://www.last.fm/music/The+Neighbourhood/_/...,https://www.last.fm/music/The+Neighbourhood,4.000000,4 мин,indie,The term 'indie' is best known as being an abb...,3711268,351516624
4,The Weeknd,Starboy,145.0,1,40,1963,5125218,3385509594,2172688,30761205,96,https://www.last.fm/music/The+Weeknd/_/Starboy,https://www.last.fm/music/The+Weeknd,3.850000,3 мин 51 сек,r&b,R&B is a musical genre that began to develop a...,5050479,1031011466
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
907,TLC,No Scrubs,0.0,178,1,1,254220,254220,1693347,12231051,970,https://www.last.fm/music/TLC/_/No+Scrubs,https://www.last.fm/music/TLC,4.033333,4 мин 02 сек,r&b,R&B is a musical genre that began to develop a...,2529703,31433334
908,Lana Del Rey,Art Deco,0.0,193,1,1,226400,226400,990127,14523928,758,https://www.last.fm/music/Lana+Del+Rey/_/Art+Deco,https://www.last.fm/music/Lana+Del+Rey,4.916667,4 мин 55 сек,indie,The term 'indie' is best known as being an abb...,5015086,1452606942
909,Katy Perry,Teenage Dream,0.0,134,1,1,186054,186054,2138230,22714306,286,https://www.last.fm/music/Katy+Perry/_/Teenage...,https://www.last.fm/music/Katy+Perry,3.783333,3 мин 47 сек,pop,Pop music is a broad-term for many different t...,6874177,414166735
910,Katy Perry,California Gurls,0.0,143,1,1,173970,173970,1553325,12237909,967,https://www.last.fm/music/Katy+Perry/_/Califor...,https://www.last.fm/music/Katy+Perry,3.916667,3 мин 55 сек,pop,Pop music is a broad-term for many different t...,6874177,414166735


In [15]:
NaN_counts = merged.isna().sum()
'''unique_artists_count = merged['Artist'].nunique()
unique_artists_count'''
NaN_counts

,0
Artist,0
Song,0
Days in top10,0
Peak,0
Days on peak,0
Days,0
PkStreams,0
Total,0
listeners,0
playcount,0


In [16]:
merged['AvgPerDay'] = merged['PkStreams'] // merged['Days on peak']
merged['Avg_log10'] = np.log10(merged['AvgPerDay'] + 1)
merged['Days_bonus'] = 1.0 + np.log10(merged['Days on peak'] + 1)
merged['Peak_bonus'] = 1.0 + (200.0 - merged['Peak']) / 200.0
merged['Score'] = merged['Avg_log10'] * merged['Days_bonus'] * merged['Peak_bonus']
merged['Score_norm'] = (merged['Score'] - merged['Score'].min()) / (merged['Score'].max() - merged['Score'].min() + 1e-12)
merged.to_csv('merged.csv', index=False)
merged

,Artist,Song,Days in top10,Peak,Days on peak,Days,PkStreams,Total,listeners,playcount,...,genre,genre_description,artist_listeners,artist_playcount,AvgPerDay,Avg_log10,Days_bonus,Peak_bonus,Score,Score_norm
0,The Weeknd,Blinding Lights,446.0,1,82,2169,8453567,5037607641,2221161,36470257,...,r&b,R&B is a musical genre that began to develop a...,5050479,1031011466,103092,5.013229,2.919078,1.995,29.194845,0.892625
1,Harry Styles,As It Was,426.0,1,81,1272,16103849,3999274385,1867858,39479081,...,pop,Pop music is a broad-term for many different t...,3037938,381185399,198812,5.298445,2.913814,1.995,30.800170,0.957736
2,Post Malone,Sunflower - Spider-Man: Into the Spider-Verse,213.0,1,16,2263,5033261,3554729660,1659244,24440448,...,rap,"Rap is a vocal style, usually coming together ...",3354307,312638678,314578,5.497730,2.230449,1.995,24.463499,0.700724
3,The Neighbourhood,Sweater Weather,0.0,12,1,1988,3342205,3532412627,2750909,45328300,...,indie,The term 'indie' is best known as being an abb...,3711268,351516624,3342205,6.524033,1.301030,1.940,16.466648,0.376376
4,The Weeknd,Starboy,145.0,1,40,1963,5125218,3385509594,2172688,30761205,...,r&b,R&B is a musical genre that began to develop a...,5050479,1031011466,128130,5.107654,2.612784,1.995,26.623667,0.788339
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
907,TLC,No Scrubs,0.0,178,1,1,254220,254220,1693347,12231051,...,r&b,R&B is a musical genre that began to develop a...,2529703,31433334,254220,5.405211,1.301030,1.110,7.805900,0.025101
908,Lana Del Rey,Art Deco,0.0,193,1,1,226400,226400,990127,14523928,...,indie,The term 'indie' is best known as being an abb...,5015086,1452606942,226400,5.354878,1.301030,1.035,7.210697,0.000960
909,Katy Perry,Teenage Dream,0.0,134,1,1,186054,186054,2138230,22714306,...,pop,Pop music is a broad-term for many different t...,6874177,414166735,186054,5.269641,1.301030,1.330,9.118429,0.078337
910,Katy Perry,California Gurls,0.0,143,1,1,173970,173970,1553325,12237909,...,pop,Pop music is a broad-term for many different t...,6874177,414166735,173970,5.240477,1.301030,1.285,8.761153,0.063846
